In [1]:
## Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import os, alive_progress
from alive_progress import alive_bar

from itertools import combinations
from scipy.stats import mannwhitneyu as mwu
from utils import VD_A as vda
from utils import apfd
from utils import cartesian
from utils import derive_data
from utils import sort_vda
from utils import calc_s12


# set inline print
%matplotlib inline

os.makedirs(f'data/img/', exist_ok=True)

# Load CSV, calculate APFD and filter Equivalent==OK

In [2]:
df_fixed = pd.concat([pd.read_csv(os.path.join(pth)) for pth in ["k_1_k_2_hads_hsi_w_wp_fixed.csv", "k_1_k_2_spy_spyh.csv"]])
df_fixed = derive_data(df_fixed)
df_fixed.EquivalenceOracle=df_fixed.CTT
df_fixed['Type'] = 'Fixed'

df_random = pd.read_csv(os.path.join("random_logs.csv"))
df_random = derive_data(df_random)
df_random.EquivalenceOracle=df_random.apply(lambda x: x.CTT+','+str(x['Random Infix Length']), axis=1)
df_random['Type'] = 'Random'
df_random['EquivalenceOracle'] = pd.Categorical(df_random['EquivalenceOracle'], 
  [f'{x[0]},{x[1]}' for x in list(cartesian((["HadsInt", "Hsi", "Wp", "W"], df_random['Random Infix Length'].unique()))
)])

# Concatenate dataframes and derive AFPD

In [3]:
equivs_drv = pd.concat([df_fixed,df_random])
equivs_drv = pd.merge(equivs_drv, pd.read_csv(os.path.join("SUL_list.csv")), how='left',on='SUL name')
equivs_drv = equivs_drv.query(f'`Equivalent`=="OK" and `Extra States`==2')

# Draw boxplots for TQ and APFD

In [4]:
dict_of_metrics = {'all':['TQ_s1', 'APFD_s1','TQ_s2',  'APFD_s2']}

for ctt_mode in equivs_drv.Type.unique():
    dataset = equivs_drv.query(f'`Type`=="{ctt_mode}"')
    metrics_s12 = calc_s12(dataset).reset_index()
    for metric_name,metrics_to_plot in dict_of_metrics.items():
        # initialize figure with 2 subplots in a row
        fig, ax = plt.subplots(2, int(len(metrics_to_plot)/2), figsize=(12,10), dpi=400)

        # add padding between the subplots
        plt.subplots_adjust(wspace=0.2,hspace=0.3)

        idx_inc=0
        for metric in metrics_to_plot:
            idx=int(idx_inc%2),int(idx_inc/2)
            # draw plots
            sns.boxplot(data=metrics_s12, ax=ax[idx], x='EquivalenceOracle',y=metric)
            ax[idx].set_xlabel('')
            #ax[idx].set_xlim([0,1])
            ax[idx].tick_params(axis='x', rotation=45)
            if "_s1" in metric and not "APFD_" in metric: ax[idx].set(yscale='log')
            idx_inc=idx_inc+1
        ##add overall title to replot
        #fig.suptitle(f'{ctt_mode} mode')
        
        # save line chart
        fig.savefig(f'data/{ctt_mode}_{metric_name}.jpg', bbox_inches='tight')
        fig.clf()

<Figure size 4800x4000 with 0 Axes>

<Figure size 4800x4000 with 0 Axes>

# Write tables for TQ and APFD

In [5]:
for name in equivs_drv.Type.unique():
    dataset = equivs_drv.query(f'`Type`=="{name}"')\
                .sort_values(['SUL name','EquivalenceOracle','Seed'])\
                .copy()
    metrics_s12 = calc_s12(dataset).reset_index()
    for metric_name,metrics_to_plot in dict_of_metrics.items():
        for metric in metrics_to_plot:
            results=metrics_s12.sort_values(['SUL name','EquivalenceOracle'])\
                        .groupby(['EquivalenceOracle'])\
                        .apply(lambda x: x[metric].tolist())\
                        .reset_index().sort_values(['EquivalenceOracle'],ascending=False)
            results=results.reindex(index=results.index[::-1])
            # apply combination method
            results = dict(zip(list(combinations(results['EquivalenceOracle'], 2)),list(combinations(results[0], 2))))
            results = pd.DataFrame.from_dict(results, orient='index').reset_index()
            results['mwu'] = results.apply(lambda x: mwu(x[0],x[1]).pvalue,axis=1)
            results['mwu<0.05'] = results['mwu']<0.05
            results[['vda_estimate','vda_magnitude']] = results.apply(lambda x: pd.Series(vda(x[0],x[1])),axis=1)
            results[['A','B']] = results['index'].apply(lambda x: pd.Series([x[0],x[1]]))
            results.drop([0,1,'index'],axis=1,inplace=True)
            results.set_index(['A','B'],inplace=True)
            results.columns.name = f'{metric} ({name})'
            results.to_csv(f'data/{metric}_{name}.csv', float_format='%.4f')
            results.style.to_latex(f'data/{metric}_{name}.tex', column_format='%.4f')

# Plot %states detected per test case (for all methods)

In [6]:
all_qtype = ['Testing symbols'] # alternative
all_runs = equivs_drv[['SUL name']].drop_duplicates()
total = len(all_qtype)*len(all_runs)

# define figure size
sns.set(rc={'figure.figsize':(10,5),'figure.dpi':300})

with alive_bar(total, force_tty=True, title=f'Plotting APFD') as bar:
    for idx,row in all_runs.iterrows():
        # get an entry <SUL, seed>
        sulname = row['SUL name']
        subj=equivs_drv.query(f'`SUL name`=="{sulname}" and `Type`=="Fixed"').copy()
        
        # add percent columns
        subj['HypSizePercent'] = subj['HypSize'].apply(lambda x: x/np.max(x)*100)

        # explode column with % of symbols and hypothesis sizes in the learning process
        subj=subj.explode(['HypSizePercent',*all_qtype])

        for qtype in all_qtype:
            #create line chart
            apfd_plot = sns.lineplot(subj, x=f'{qtype}', y='HypSizePercent',
                                     markers=True, 
                                     style='CTT', hue='CTT',
                                     palette='tab10'
                                    )
            apfd_plot.set(xscale='log')
            locator = ticker.LogLocator()
            locator.MAXTICKS = np.max(subj[f'{qtype}'])
            apfd_plot.xaxis.set_major_locator(locator)

            apfd_plot.yaxis.set_major_locator(ticker.MultipleLocator(10))
            apfd_plot.set_ylim(0,100)
            
            #add plot labels, titles and legends
            plt.xlabel(f'Number of {qtype.title()} (log scale)')
            plt.ylabel('Fraction of the SUL learned')
            plt.title(f'Subject: {sulname}')
            
            #get handles and labels
            handles, labels = plt.gca().get_legend_handles_labels()
            
            #specify order of items in legend from APFD
            order = subj[['CTT','APFD']].sort_values(['APFD'],ascending=False).drop_duplicates().CTT.to_list()
            for idx,ctt in enumerate(order): order[idx]=labels.index(ctt)
            #add legend to plot
            plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],
                       title='Testing Technique', loc='lower right', 
                       fontsize='xx-small', title_fontsize='xx-small')
            
            # save line chart
            fig = apfd_plot.get_figure()
            fname=sulname.replace('.dot','')
            fig.savefig(f'data/img/{fname}_{qtype}.jpg', dpi=400)
            fig.clf()
            bar()

Plotting APFD |████████████████████████████████████████| 46/46 [100%] in 25.9s (1.76/s)                                 


<Figure size 3000x1500 with 0 Axes>